### What differs from Darts implementation
- more feature engineered
- target y: units sold for one week/ sum of 4 weeks

### Prepare dataset

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

In [2]:
df = pd.read_csv('../data/retail_store_inventory.csv')
df['Date'] = pd.to_datetime(df['Date'])

df.drop(['Store ID', 'Category', 'Region', 'Inventory Level', 'Units Ordered', 'Demand Forecast', 'Price', 'Discount', 'Competitor Pricing'], axis=1, inplace=True)

# Aggregate across all stores/ regions/ categories
aggregated_df = df.groupby(['Date', 'Product ID']).agg({ # Keep unique Date and Product ID pairs
    'Units Sold': 'sum',
    'Weather Condition': lambda x: x.mode()[0] if not x.mode().empty else None,  # Mode
    'Holiday/Promotion': 'max',  # If any promotion/holiday exists, it is recorded
    'Seasonality': lambda x: x.mode()[0] if not x.mode().empty else None  # Mode
}).reset_index()

# Aggregate on weekly baisis
aggregated_df['Week'] = aggregated_df['Date'].dt.to_period('W').astype(str)  # Create a 'Week' column based on the ISO calendar week

weekly_aggregated_df = aggregated_df.groupby(['Week', 'Product ID']).agg({
    'Units Sold': 'sum',
    'Weather Condition': lambda x: x.mode()[0] if not x.mode().empty else None,  # Mode
    'Holiday/Promotion': 'max',  # If any promotion/holiday exists, it is recorded
    'Seasonality': lambda x: x.mode()[0] if not x.mode().empty else None  # Mode
}).reset_index()

# Reformat the 'Week' column
weekly_aggregated_df['Week'] = pd.to_datetime(weekly_aggregated_df['Week'].str.split('/').str[0])  # Takes the first date of the week
weekly_aggregated_df = weekly_aggregated_df.sort_values(by=['Product ID', 'Week']) # Sort for consistency

# Encoding categorical variables as numerical for Darts
weekly_aggregated_df['Weather Condition'] = weekly_aggregated_df['Weather Condition'].astype('category').cat.codes
weekly_aggregated_df['Seasonality'] = weekly_aggregated_df['Seasonality'].astype('category').cat.codes

In [ ]:
df = weekly_aggregated_df

df = df.sort_values(by=["Product ID", "Week"])

df["Product ID"] = LabelEncoder().fit_transform(df["Product ID"]) # Encode Product ID

df.head()

,Week,Product ID,Units Sold,Weather Condition,Holiday/Promotion,Seasonality
0,2021-12-27,P0001,1353,0,1,0
20,2022-01-03,P0001,4606,1,1,2
40,2022-01-10,P0001,3557,1,1,3
60,2022-01-17,P0001,4920,1,1,1
80,2022-01-24,P0001,5003,2,1,0


### Feature Engineering + 1-week Y

In [ ]:
# # --- Feature engineering function ---
# def create_features(df, lags=[1, 2, 3, 4], windows=[4, 8]):
#     features = []
#     for pid, group in df.groupby("Product ID"):
#         group = group.copy().sort_values("Week")
        
#         for lag in lags:
#             group[f"lag_{lag}"] = group["Units Sold"].shift(lag)

#         for window in windows:
#             group[f"roll_mean_{window}"] = group["Units Sold"].shift(1).rolling(window).mean()
#             group[f"roll_std_{window}"] = group["Units Sold"].shift(1).rolling(window).std()
        
#         group["weekofyear"] = group["Week"].dt.isocalendar().week
#         group["month"] = group["Week"].dt.month
#         # group["dayofweek"] = group["Week"].dt.dayofweek
#         group["is_month_start"] = group["Week"].dt.is_month_start.astype(int)
#         group["is_month_end"] = group["Week"].dt.is_month_end.astype(int)
#         group["diff_1"] = group["Units Sold"].diff(1)
        
#         features.append(group.dropna().reset_index(drop=True))

#     return pd.concat(features, ignore_index=True)

# df_feat = create_features(df)
# df_feat.head()

,Week,Product ID,Units Sold,Weather Condition,Holiday/Promotion,Seasonality,lag_1,lag_2,lag_3,lag_4,roll_mean_4,roll_std_4,roll_mean_8,roll_std_8,weekofyear,month,is_month_start,is_month_end,diff_1
0,2022-02-21,0,4225,1,1,0,4797.0,4142.0,5253.0,5003.0,4798.75,475.879799,4203.875,1271.408941,8,2,0,0,-572.0
1,2022-02-28,0,5500,2,1,1,4225.0,4797.0,4142.0,5253.0,4604.25,521.387492,4562.875,555.141536,9,2,0,1,1275.0
2,2022-03-07,0,4869,1,1,0,5500.0,4225.0,4797.0,4142.0,4666.00,627.634182,4674.625,647.380863,10,3,0,0,-631.0
3,2022-03-14,0,3509,2,1,0,4869.0,5500.0,4225.0,4797.0,4847.75,521.623987,4838.625,464.026150,11,3,0,0,-1360.0
4,2022-03-21,0,4652,1,1,0,3509.0,4869.0,5500.0,4225.0,4525.75,854.637301,4662.250,656.794978,12,3,0,0,1143.0


In [ ]:
# # --- Select features ---
# target_col = "Units Sold"
# drop_cols = ["Week", target_col]
# feature_cols = [col for col in df_feat.columns if col not in drop_cols]

#### Predict future 1 week

In [ ]:
# # --- Rolling backtest function (local)---
# def local_rolling_backtest(df, feature_cols, target_col="Units Sold", train_frac=0.7):
#     df = df.sort_values(["Product ID", "Week"]).reset_index(drop=True)
#     all_preds = []
#     all_truths = []

#     for pid, group in tqdm(df.groupby("Product ID")):
#         group = group.sort_values("Week")
#         n = len(group)
#         split = int(n * train_frac)

#         for i in range(split, n):
#             train = group.iloc[:i]
#             test = group.iloc[i:i+1]

#             if len(test) == 0: continue

#             model = RandomForestRegressor(n_estimators=100, random_state=42)
#             model.fit(train[feature_cols], train[target_col])
#             pred = model.predict(test[feature_cols])
#             all_preds.append(pred[0])
#             all_truths.append(test[target_col].values[0])

#     mape = mean_absolute_percentage_error(all_truths, all_preds)
#     return mape, all_preds, all_truths

# # --- Run backtest ---
# mape, y_pred, y_true = local_rolling_backtest(df_feat, feature_cols)

# print(f"Overall MAPE (Global Model with Backtesting): {mape:.4f}")

100%|██████████| 20/20 [00:31<00:00,  1.55s/it]

Overall MAPE (Global Model with Backtesting): 0.2690


In [ ]:
# # --- Rolling backtest function (global)---
# def global_rolling_backtest(df, feature_cols, target_col="Units Sold", train_frac=0.7):
#     df = df.sort_values(["Week", "Product ID"]).reset_index(drop=True)
#     all_preds = []
#     all_truths = []

#     # Convert Week to numeric index for easy slicing
#     unique_weeks = sorted(df["Week"].unique())
#     split_idx = int(len(unique_weeks) * train_frac)

#     for i in range(split_idx, len(unique_weeks)):
#         train_weeks = unique_weeks[:i]
#         test_week = unique_weeks[i]

#         train = df[df["Week"].isin(train_weeks)]
#         test = df[df["Week"] == test_week]

#         if len(test) == 0 or len(train) == 0:
#             continue

#         model = RandomForestRegressor(n_estimators=100, random_state=42)
#         model.fit(train[feature_cols], train[target_col])
#         preds = model.predict(test[feature_cols])

#         all_preds.extend(preds)
#         all_truths.extend(test[target_col].values)

#     mape = mean_absolute_percentage_error(all_truths, all_preds)
#     return mape, all_preds, all_truths

# # --- Run backtest ---
# mape, y_pred, y_true = global_rolling_backtest(df_feat, feature_cols)
# print(f"Global Model MAPE with Rolling Backtest: {mape:.4f}")

Global Model MAPE with Rolling Backtest: 0.0085


##### Predict future 4 weeks -- Recursive
- Predict week t+1, then use it as input to predict t+2, and so on.
- Updates lag features using predicted values (not true ones) in later steps.
- Simple but can suffer from error propagation.

In [15]:
def recursive_forecast(df_last_window, model, feature_cols, forecast_horizon=4):
    """
    Predict next N steps recursively using a 1-step-ahead trained model.
    df_last_window: the last few rows (e.g. last 3 weeks) for a product
    model: trained regression model (1-step ahead)
    feature_cols: columns to use for prediction
    """
    preds = []
    df_current = df_last_window.copy()

    for step in range(forecast_horizon):
        # Prepare input for prediction
        X_input = df_current.iloc[[-1]][feature_cols]
        y_pred = model.predict(X_input)[0]
        preds.append(y_pred)

        # Create a new row (simulate next week)
        new_week = df_current["Week"].max() + pd.Timedelta(weeks=1)
        new_row = df_current.iloc[[-1]].copy()
        new_row["Week"] = new_week
        new_row["Units Sold"] = y_pred  # fake target

        # Shift lag features manually
        for lag in [1, 2, 3, 4]:  
            if f"lag_{lag}" in new_row.columns:
                if lag == 1:
                    new_row[f"lag_{lag}"] = df_current.iloc[-1]["Units Sold"]
                else:
                    new_row[f"lag_{lag}"] = df_current.iloc[-lag]["Units Sold"] if len(df_current) >= lag else np.nan

        # Rolling features — recompute using history + prediction
        for window in [4, 8]:
            past_vals = pd.concat([df_current["Units Sold"], pd.Series(preds)])[-window:]
            new_row[f"roll_mean_{window}"] = past_vals.mean()
            new_row[f"roll_std_{window}"] = past_vals.std()

        # Time features
        new_row["weekofyear"] = new_week.isocalendar().week
        new_row["month"] = new_week.month
        # new_row["dayofweek"] = new_week.dayofweek
        new_row["is_month_start"] = int(new_week.is_month_start)
        new_row["is_month_end"] = int(new_week.is_month_end)

        # Differenced value — assume diff from last real/forecast value
        new_row["diff_1"] = y_pred - df_current.iloc[-1]["Units Sold"]

        # Append to df
        df_current = pd.concat([df_current, new_row], ignore_index=True)

    return preds

In [16]:
def global_recursive_backtest(df, feature_cols, target_col="Units Sold", forecast_horizon=4, train_frac=0.7):
    df = df.sort_values(["Week", "Product ID"]).reset_index(drop=True)
    all_preds = []
    all_truths = []

    unique_weeks = sorted(df["Week"].unique())
    split_idx = int(len(unique_weeks) * train_frac)

    for i in tqdm(range(split_idx, len(unique_weeks) - forecast_horizon + 1)):
        train_weeks = unique_weeks[:i]
        test_weeks = unique_weeks[i:i + forecast_horizon]

        train = df[df["Week"].isin(train_weeks)]
        test = df[df["Week"].isin(test_weeks)]

        # Train one-step model on all data
        model = RandomForestRegressor(n_estimators=100, random_state=42)
        model.fit(train[feature_cols], train[target_col])

        # Predict recursively per product
        for pid, test_group in df[df["Week"] == test_weeks[0]].groupby("Product ID"):
            history = df[(df["Product ID"] == pid) & (df["Week"].isin(train_weeks))].copy()
            history = history.sort_values("Week")

            if len(history) < 5:  # ensure enough for lags
                continue

            df_last_window = history.iloc[-5:]  # enough for lag and rolling
            pred_4wk = recursive_forecast(df_last_window, model, feature_cols, forecast_horizon=forecast_horizon)

            # Actual 4-week future sum
            actual_sum = test[(test["Product ID"] == pid)]["Units Sold"].sum()

            if len(pred_4wk) == forecast_horizon:
                all_preds.append(sum(pred_4wk))
                all_truths.append(actual_sum)

    mape = mean_absolute_percentage_error(all_truths, all_preds)
    return mape, all_preds, all_truths

mape_rec, y_pred_rec, y_true_rec = global_recursive_backtest(df_feat, feature_cols)
print(f"Recursive Forecast MAPE (4-week sum): {mape_rec:.4f}")

100%|██████████| 27/27 [00:25<00:00,  1.06it/s]

Recursive Forecast MAPE (4-week sum): 0.1162


##### Predict future 4 weeks -- Multi-output
- Train model to output multiple steps in one go: y = [y_t+1, y_t+2, y_t+3, y_t+4]
- Requires reshaping your target to a 4-dimensional vector.
- Avoids compounding error but needs more data to learn joint patterns.

Notes on MultiOutputRegressor
- A wrapper in sklearn around any base regressor
- It clones one independent model for each target output.
- Then it fits each column separately using a copy of the base model

In [10]:
# Feature engineering: prepare 4-step target columns
from sklearn.multioutput import MultiOutputRegressor

def create_multi_output_features(df, lags=[1, 2, 3], windows=[3, 5], forecast_horizon=4):
    features = []
    for pid, group in df.groupby("Product ID"):
        group = group.copy().sort_values("Week")

        # Lag features
        for lag in lags:
            group[f"lag_{lag}"] = group["Units Sold"].shift(lag)

        # Rolling statistics
        for window in windows:
            group[f"roll_mean_{window}"] = group["Units Sold"].shift(1).rolling(window).mean()
            group[f"roll_std_{window}"] = group["Units Sold"].shift(1).rolling(window).std()

        # Date/time features
        group["weekofyear"] = group["Week"].dt.isocalendar().week
        group["month"] = group["Week"].dt.month
        group["dayofweek"] = group["Week"].dt.dayofweek
        group["is_month_start"] = group["Week"].dt.is_month_start.astype(int)
        group["is_month_end"] = group["Week"].dt.is_month_end.astype(int)

        # Difference
        group["diff_1"] = group["Units Sold"].diff(1)

        # Multi-output target columns
        for step in range(1, forecast_horizon + 1):
            group[f"y_t+{step}"] = group["Units Sold"].shift(-step)

        features.append(group.dropna().reset_index(drop=True))

    return pd.concat(features, ignore_index=True)


# Feature engineering
df_feat_multi = create_multi_output_features(df)

# Define input and output
target_cols = [f"y_t+{i}" for i in range(1, 5)]
drop_cols = ["Week", "Units Sold"] + target_cols
feature_cols = [col for col in df_feat_multi.columns if col not in drop_cols]

df_feat_multi.head()

,Week,Product ID,Units Sold,Weather Condition,Holiday/Promotion,Seasonality,lag_1,lag_2,lag_3,roll_mean_3,...,weekofyear,month,dayofweek,is_month_start,is_month_end,diff_1,y_t+1,y_t+2,y_t+3,y_t+4
0,2022-01-31,0,5253,0,1,1,5003.0,4920.0,3557.0,4493.333333,...,5,1,0,0,1,250.0,4142.0,4797.0,4225.0,5500.0
1,2022-02-07,0,4142,1,1,1,5253.0,5003.0,4920.0,5058.666667,...,6,2,0,0,0,-1111.0,4797.0,4225.0,5500.0,4869.0
2,2022-02-14,0,4797,1,1,2,4142.0,5253.0,5003.0,4799.333333,...,7,2,0,0,0,655.0,4225.0,5500.0,4869.0,3509.0
3,2022-02-21,0,4225,1,1,0,4797.0,4142.0,5253.0,4730.666667,...,8,2,0,0,0,-572.0,5500.0,4869.0,3509.0,4652.0
4,2022-02-28,0,5500,2,1,1,4225.0,4797.0,4142.0,4388.000000,...,9,2,0,0,1,1275.0,4869.0,3509.0,4652.0,3812.0


In [11]:
def global_multioutput_backtest(df, feature_cols, target_cols, train_frac=0.7):
    df = df.sort_values(["Week", "Product ID"]).reset_index(drop=True)
    all_preds = []
    all_truths = []

    unique_weeks = sorted(df["Week"].unique())
    split_idx = int(len(unique_weeks) * train_frac)

    for i in tqdm(range(split_idx, len(unique_weeks))):
        train_weeks = unique_weeks[:i]
        test_week = unique_weeks[i]

        train = df[df["Week"].isin(train_weeks)]
        test = df[df["Week"] == test_week]

        if train.empty or test.empty:
            continue

        model = MultiOutputRegressor(RandomForestRegressor(n_estimators=100, random_state=42))
        model.fit(train[feature_cols], train[target_cols])

        preds = model.predict(test[feature_cols])
        preds_sum = preds.sum(axis=1)  # sum of 4 steps for each sample

        truths_sum = test[target_cols].sum(axis=1)  # sum of actual 4-week ahead

        all_preds.extend(preds_sum)
        all_truths.extend(truths_sum)

    mape = mean_absolute_percentage_error(all_truths, all_preds)
    return mape, all_preds, all_truths

mape_multi, y_pred_multi, y_true_multi = global_multioutput_backtest(df_feat_multi, feature_cols, target_cols)
print(f"Multi-Output Direct Model MAPE (4-week sum): {mape_multi:.4f}")

100%|██████████| 30/30 [01:37<00:00,  3.26s/it]

Multi-Output Direct Model MAPE (4-week sum): 0.0646


##### Predict future 4 weeks -- Ensemble
- Train 4 separate models, one for each horizon (1-week ahead, 2-week ahead, …)
- Each model specializes in a fixed forecast horizon.
- Often outperforms recursive but costs more to train and tune.

In [ ]:
# # Feature engineering (already done if you ran earlier step)
# df_feat_multi = create_multi_output_features(df)

# # Feature and target columns
# target_cols = [f"y_t+{i}" for i in range(1, 5)]
# drop_cols = ["Week", "Units Sold"] + target_cols
# feature_cols = [col for col in df_feat_multi.columns if col not in drop_cols]

In [12]:
def global_ensemble_backtest(df, feature_cols, target_cols, train_frac=0.7):
    df = df.sort_values(["Week", "Product ID"]).reset_index(drop=True)
    all_preds = []
    all_truths = []

    unique_weeks = sorted(df["Week"].unique())
    split_idx = int(len(unique_weeks) * train_frac)

    for i in tqdm(range(split_idx, len(unique_weeks))):
        train_weeks = unique_weeks[:i]
        test_week = unique_weeks[i]

        train = df[df["Week"].isin(train_weeks)]
        test = df[df["Week"] == test_week]

        if train.empty or test.empty:
            continue

        models = {}
        for idx, target in enumerate(target_cols):  # create a model for each target
            model = RandomForestRegressor(n_estimators=100, random_state=42)
            model.fit(train[feature_cols], train[target])
            models[target] = model

        preds = []
        for target, model in models.items():
            pred = model.predict(test[feature_cols])
            preds.append(pred)

        preds = np.vstack(preds).T  # shape: (n_samples, 4)

        preds_sum = preds.sum(axis=1)
        truths_sum = test[target_cols].sum(axis=1)

        all_preds.extend(preds_sum)
        all_truths.extend(truths_sum)

    mape = mean_absolute_percentage_error(all_truths, all_preds)
    return mape, all_preds, all_truths

mape_ensemble, y_pred_ensemble, y_true_ensemble = global_ensemble_backtest(df_feat_multi, feature_cols, target_cols)
print(f"Direct Ensemble Model MAPE (4-week sum): {mape_ensemble:.4f}")

100%|██████████| 30/30 [01:37<00:00,  3.26s/it]

Direct Ensemble Model MAPE (4-week sum): 0.0646


### Feature Engineering + 4-week Y

In [22]:
def create_features_with_rolling_target(df, lags=[1, 2, 3, 4], windows=[4, 8], forecast_horizon=4):
    features = []
    for pid, group in df.groupby("Product ID"):
        group = group.copy().sort_values("Week")

        # Lag features
        for lag in lags:
            group[f"lag_{lag}"] = group["Units Sold"].shift(lag)

        # Rolling stats
        for window in windows:
            group[f"roll_mean_{window}"] = group["Units Sold"].shift(1).rolling(window).mean()
            group[f"roll_std_{window}"] = group["Units Sold"].shift(1).rolling(window).std()

        # Date/time features
        group["weekofyear"] = group["Week"].dt.isocalendar().week
        group["month"] = group["Week"].dt.month
        # group["dayofweek"] = group["Week"].dt.dayofweek
        group["is_month_start"] = group["Week"].dt.is_month_start.astype(int)
        group["is_month_end"] = group["Week"].dt.is_month_end.astype(int)

        # Differenced target
        group["diff_1"] = group["Units Sold"].diff(1)

        # New target: future 4-week sum
        group["future_4wk_sum"] = group["Units Sold"].shift(-forecast_horizon + 1).rolling(forecast_horizon).sum()

        features.append(group.dropna().reset_index(drop=True))

    return pd.concat(features, ignore_index=True)

df_feat = create_features_with_rolling_target(df)
df_feat.head()

,Week,Product ID,Units Sold,Weather Condition,Holiday/Promotion,Seasonality,lag_1,lag_2,lag_3,lag_4,roll_mean_4,roll_std_4,roll_mean_8,roll_std_8,weekofyear,month,is_month_start,is_month_end,diff_1,future_4wk_sum
0,2022-02-21,0,4225,1,1,0,4797.0,4142.0,5253.0,5003.0,4798.75,475.879799,4203.875,1271.408941,8,2,0,0,-572.0,18103.0
1,2022-02-28,0,5500,2,1,1,4225.0,4797.0,4142.0,5253.0,4604.25,521.387492,4562.875,555.141536,9,2,0,1,1275.0,18530.0
2,2022-03-07,0,4869,1,1,0,5500.0,4225.0,4797.0,4142.0,4666.00,627.634182,4674.625,647.380863,10,3,0,0,-631.0,16842.0
3,2022-03-14,0,3509,2,1,0,4869.0,5500.0,4225.0,4797.0,4847.75,521.623987,4838.625,464.026150,11,3,0,0,-1360.0,16551.0
4,2022-03-21,0,4652,1,1,0,3509.0,4869.0,5500.0,4225.0,4525.75,854.637301,4662.250,656.794978,12,3,0,0,1143.0,18676.0


In [23]:
# Define target and features
target_col = "future_4wk_sum"
drop_cols = ["Week", "Units Sold", "future_4wk_sum"]
feature_cols = [col for col in df_feat.columns if col not in drop_cols]

In [ ]:
def global_rolling_backtest(df, feature_cols, target_col, train_frac=0.7):
    df = df.sort_values(["Week", "Product ID"]).reset_index(drop=True)
    all_preds = []
    all_truths = []

    # Convert Week to numeric index for easy slicing
    unique_weeks = sorted(df["Week"].unique())
    split_idx = int(len(unique_weeks) * train_frac)

    for i in range(split_idx, len(unique_weeks)):
        train_weeks = unique_weeks[:i]
        test_week = unique_weeks[i]

        train = df[df["Week"].isin(train_weeks)]
        test = df[df["Week"] == test_week]

        if len(test) == 0 or len(train) == 0:
            continue

        model = RandomForestRegressor(n_estimators=100, random_state=42)
        model.fit(train[feature_cols], train[target_col])
        preds = model.predict(test[feature_cols])

        all_preds.extend(preds)
        all_truths.extend(test[target_col].values)

    mape = mean_absolute_percentage_error(all_truths, all_preds)
    return mape, all_preds, all_truths


In [31]:
mape_direct, y_pred_direct, y_true_direct = global_rolling_backtest(df_feat, feature_cols, target_col="future_4wk_sum")
print(f"Global Model MAPE (4-week sum forecast): {mape_direct:.4f}")

Global Model MAPE (4-week sum forecast): 0.0589


### Compare

In [30]:
print(f"MAPE (Recursive): {mape_rec*100:.3f}%")
print(f"MAPE (Multi-output): {mape_multi*100:.3f}%")
print(f"MAPE (Ensemble): {mape_ensemble*100:.3f}%")
print(f"MAPE (Direct): {mape_direct*100:.3f}%")


MAPE (Recursive): 11.621%
MAPE (Multi-output): 6.463%
MAPE (Ensemble): 6.463%
MAPE (Direct): 5.893%


In this case, Multi-output is essentially the same as Ensemble due to same hyperparameter for each model in Ensemble.

Different from truly multi-output model like
- Neural Network - last layer output a vector at once
- Seq2Seq models - output a sequence by decoding step-by-step, with previous outputs fed back in 